In [1]:
pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 7.8 MB/s 
     |████████████████████████████████| 462 kB 57.7 MB/s 


In [2]:
import nltk
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [5]:
df = pd.read_csv("Miami_Dataset2.0.csv",encoding='cp1252')
df

,Transcription,Truthful
0,My best friend is a really nice person. Um. Sh...,1
1,She’s actually really two faced and not fun to...,0
2,So this specific person is actually just a rea...,1
3,This person is actually a really kind person. ...,0
4,My roommate is a really good person. She’s ver...,0
...,...,...
315,He’s a really good guy. Uh. He’s always cleani...,1
316,My roommate is a great guy. There is many grea...,0
317,"My roommate is insufferable. He is, uh, comple...",0
318,My cousin is a terrible person. She makes me r...,0


In [ ]:
#df["Statement"] = df["Statement"].apply(lambda x: x.replace("ammm", ""))
#df["Statement"] = df["Statement"].apply(lambda x: x.replace(" � ", ""))
#df["Statement"] = df["Statement"].apply(lambda x: x.replace("...", ""))
#df["Statement"] = df["Statement"].apply(lambda x: x.replace("um", ""))
#df["Statement"] = df["Statement"].apply(lambda x: x.replace("Um", ""))
#df["Statement"] = df["Statement"].apply(lambda x: x.replace(" � ", ""))
#df["Statement"] = df["Statement"].apply(lambda x: x.replace("Uh", ""))
#df["Statement"] = df["Statement"].apply(lambda x: x.replace("oh,", ""))


In [ ]:
#import re
#df.loc[:,"Statement"] = df.Statement.apply(lambda x : " ".join(re.findall('[\w]+',x)))

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
stop_words = set(stopwords.words("english"))

def remove_stopWords(s):
 
    s = ' '.join(word for word in s.split() if word not in stop_words)
    return s

df.loc[:,"Transcription"] = df.Transcription.apply(lambda x: remove_stopWords(x))

In [13]:
sentences= df['Transcription']

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
tokenized_sent = []
for s in sentences:
    tokenized_sent.append(word_tokenize(s.lower()))
tokenized_sent

[['my',
  'best',
  'friend',
  'really',
  'nice',
  'person',
  '.',
  'um',
  '.',
  'she',
  '’',
  's',
  'always',
  'kind',
  'everyone',
  '.',
  'she',
  'continues',
  'around',
  'everyone',
  '.',
  'um',
  '.',
  'she',
  'taught',
  'much',
  'throughout',
  ',',
  'like',
  'i',
  '’',
  've',
  'known',
  'maybe',
  'year',
  'half',
  'she',
  '’',
  's',
  'taught',
  'much',
  'throughout',
  'time',
  '.',
  'um',
  '.',
  'i',
  'love',
  'much',
  'she',
  '’',
  's',
  'influenced',
  'life',
  'many',
  'ways',
  '.',
  'and',
  'made',
  'better',
  'person',
  'she',
  '’',
  's',
  'nice',
  'pershon',
  ',',
  'person',
  '.',
  'and',
  'i',
  'wish',
  'uh',
  'people',
  'world',
  'like',
  'um',
  'people',
  'need',
  'realize',
  'negative',
  'time',
  'isn',
  '’',
  't',
  'helping',
  'all',
  '.',
  'so',
  ',',
  'she',
  '’',
  's',
  'awesome',
  '.'],
 ['she',
  '’',
  's',
  'actually',
  'really',
  'two',
  'faced',
  'fun',
  'around',
  

In [17]:
df.groupby('Truthful').describe()

Transcription         \
                 count unique   
Truthful                        
0                  195    195   
1                  125    125   

                                                                  
                                                        top freq  
Truthful                                                          
0         She’s actually really two faced fun around. Um...    1  
1         My best friend really nice person. Um. She’s a...    1

In [18]:
df['Truthful'].value_counts()

0    195
1    125
Name: Truthful, dtype: int64

In [19]:
df_d = df[df['Truthful']==1]
df_d.shape

(125, 2)

In [20]:
df_t= df[df['Truthful']==0]
df_t.shape

(195, 2)

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Transcription'],df['Truthful'],test_size=0.1)

In [22]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [23]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']


In [24]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [25]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [26]:
len(X_train)

288

In [30]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [31]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
9/9 [==============================] - 5s 323ms/step - loss: 0.6975 - accuracy: 0.5764 - precision: 0.3684 - recall: 0.1250
Epoch 2/100
9/9 [==============================] - 3s 323ms/step - loss: 0.6661 - accuracy: 0.5972 - precision: 0.4167 - recall: 0.0893
Epoch 3/100
9/9 [==============================] - 3s 325ms/step - loss: 0.6796 - accuracy: 0.5868 - precision: 0.3704 - recall: 0.0893
Epoch 4/100
9/9 [==============================] - 3s 326ms/step - loss: 0.6766 - accuracy: 0.6354 - precision: 0.5897 - recall: 0.2054
Epoch 5/100
9/9 [==============================] - 3s 326ms/step - loss: 0.6717 - accuracy: 0.6111 - precision: 0.5000 - recall: 0.0536
Epoch 6/100
9/9 [==============================] - 3s 329ms/step - loss: 0.6458 - accuracy: 0.6389 - precision: 0.5952 - recall: 0.2232
Epoch 7/100
9/9 [==============================] - 3s 328ms/step - loss: 0.6471 - accuracy: 0.6215 - precision: 0.5273 - recall: 0.2589
Epoch 8/100
9/9 [==============================]

In [32]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 1s 1s/step - loss: 0.7589 - accuracy: 0.4688 - precision: 0.1667 - recall: 0.0769


[0.7588714361190796, 0.46875, 0.1666666716337204, 0.07692307978868484]